In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
from scipy.stats import gumbel_r
import pyomo.environ as pyo
from pyomo.environ import *

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def collection_distribution_hev(n,collection,price):
    # generate mean 
    ''' mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1) '''
    # generate deterministic utilities for products
    
    rho = -0.5 # prices and utilities are negatively correlated
    price_mean = np.mean(price[1:]) 
    price_std = np.std(price[1:])
    
    z = np.zeros(len(price[1:]))
    for i in range(len(z)):
        z[i] = (price[1:][i] - price_mean)/price_std
        
    mu_1n = np.zeros(len(z))
    normal_rvs = np.random.randn(len(mu_1n))
    for i in range(len(mu_1n)):
        mu_1n[i] = rho*z[i] + np.sqrt(1-rho**2)*normal_rvs[i] 
    ## deterministic utility of the outside option is strictly greater than the utilities of the products
    mu_0 = np.max(mu_1n) + 1
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = 10
    scale_1n = np.random.uniform(0.04,1,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])
    # generate covariance matrix with positive correlation
    # neg_cov = generate_negatively_correlated_covariance_matrix(n)
    # if is_positive_semidefinite(neg_cov): 
    #     sample_size = 10000
    #     utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    # else:
    #     print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [5]:
''' def collection_distribution_hev(n,collection):
    # generate mean 
    mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1)
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = np.random.uniform(9,10)
    scale_1n = np.random.uniform(0.04,2,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])

    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution '''

' def collection_distribution_hev(n,collection):\n    # generate mean \n    mu_0 = np.random.uniform(2,3)\n    mu_1n = np.random.uniform(-3,-2,n-1)\n    # np.random.uniform(-2,2,n-1)\n    mu = np.hstack((mu_0,mu_1n))\n    \n    scale_0 = np.random.uniform(9,10)\n    scale_1n = np.random.uniform(0.04,2,n-1)\n    scales = np.hstack((scale_0,scale_1n))\n    \n    sample_size = 10000\n    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])\n\n    \n    collection_distribution = np.zeros((n,len(collection)))\n    for i in range(len(collection)):\n        curr_assortment = collection[i]\n        curr_population = [[] for _ in range(sample_size)] \n\n        for j in range(sample_size):\n            for k in curr_assortment:\n                curr_population[j].append(utility_samples[j][k])\n                # each sub list records only the utilities of products in the current assortment\n                \n        frequency = [0]*len(curr_assortment)\n  

In [6]:
def whole_instance_generation(n,whole_collection,whole_offertimes,price):
    ## generate LCmccm instances based on the assortment collection infomation
    # step 1: randomly generate LCmccm parameters
    # 1.1 the number of classes in LCmccm, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mccm
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCmccm instance with the above parameters of LCmccm
    true_hev_instance = collection_distribution_hev(n,whole_collection,price)
    # step 3: generate multinomial samples based on lcmccm instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_hev_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [7]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [8]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [9]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [10]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [11]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [12]:

## generate 50 random full instances 

pred_full_instance = []
pred_full_samples = []

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])
    
    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [13]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [14]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        ''' condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-3:
                    condition = True
        if condition == True:
            print('instance_generation error') '''
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [15]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
     
        df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [18]:
def mccm_limit_kl(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    #model.abs_var = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="abs_val")

    model.obj = pyo.Objective(expr= sum( sum(offertimes[j] * data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.maximize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)

    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)

    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    #solver.options['tol'] = 1e-9
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            # Retrieve the solver runtime
            #model.pprint()
            #return [pyo.value(model.obj), x_values, lam_values, rho_values, y_values, results.solver.time]
            return [pyo.value(model.obj), x_values, results.solver.time]
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM MLE: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)),  -1]



In [19]:
## record the limit loss of all instances
all_mccm_limit_loss = []
all_mccm_limit_prob = []
all_mccm_limit_runtime = []

error_i = []
error_j = []


for i in range(len(pred_offer_times_list)):
    
    ## define container for mccm limit 
    mccm_limit_loss_collection =[]
    mccm_limit_runtime_collection =[]
    mccm_limit_probability_collection =[]
    
       
    for j in range(pred_instance_size[i]):
        print(f'testing instance with offertimes {pred_offer_times_list[i]} : {j}th ')
        
        curr_limit_result = mccm_limit_kl(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
        # return [model.objVal, model.Runtime,lam_values,x_values]
        curr_limit_prob = curr_limit_result[1]
        mccm_limit_probability_collection.append(curr_limit_prob)
        mccm_limit_loss_collection.append(curr_limit_result[0])
        mccm_limit_runtime_collection.append(curr_limit_result[-1])
        if curr_limit_result[-1]<0:
            error_i.append(i)
            error_j.append(j)
            print(f' ERROR with instance {i} {j}')
        else:
            print(f' SUCCESS instance {i} {j}')

    ## for each train instance [i][j] limit loss result is a number
    all_mccm_limit_loss.append(mccm_limit_loss_collection)
    all_mccm_limit_runtime.append(mccm_limit_runtime_collection)
    ## for each train instance [i][j] limit probability is a matrix
    all_mccm_limit_prob.append(mccm_limit_probability_collection)
    

testing instance with offertimes 20 : 0th 
 SUCCESS instance 0 0
testing instance with offertimes 20 : 1th 
Error: Cannot load a SolverResults object with bad status: error
MCCM MLE: No solution found or an error occurred.
 ERROR with instance 0 1
testing instance with offertimes 20 : 2th 
 SUCCESS instance 0 2
testing instance with offertimes 20 : 3th 
 SUCCESS instance 0 3
testing instance with offertimes 20 : 4th 
 SUCCESS instance 0 4
testing instance with offertimes 20 : 5th 
Error: Cannot load a SolverResults object with bad status: error
MCCM MLE: No solution found or an error occurred.
 ERROR with instance 0 5
testing instance with offertimes 20 : 6th 
 SUCCESS instance 0 6
testing instance with offertimes 20 : 7th 
 SUCCESS instance 0 7
testing instance with offertimes 20 : 8th 
 SUCCESS instance 0 8
testing instance with offertimes 20 : 9th 
 SUCCESS instance 0 9
testing instance with offertimes 20 : 10th 
 SUCCESS instance 0 10
testing instance with offertimes 20 : 11th 
 SU

In [20]:
print(error_i)
print(error_j)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[1, 5, 19, 23, 25, 28, 31, 32, 34, 37, 38, 47, 0, 5, 12, 19, 22, 25, 28, 31, 35, 40, 0, 4, 5, 18, 25, 28, 29, 32, 34, 39, 2, 9, 20, 30, 31, 34, 37, 49, 12, 17, 21, 22, 27, 30, 31, 33, 34, 35, 37, 39, 42, 44]


In [65]:
def mccm_limit_kl2(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
   

    model.obj = pyo.Objective(expr= sum( sum(offertimes[j] * data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.maximize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
       
            
    
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)

    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] =  10000
    solver.options['acceptable_tol'] = 1e-4  # Control acceptable convergence tolerance
    solver.options['tol'] = 1e-4  # Relax tolerance for convergence
    solver.options['dual_inf_tol'] = 1e-4  # Reduce tolerance for dual infeasibility
    
    try:

        # Your optimization solve code here
        #results = solver.solve(model, tee=True, logfile='solver.log')
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            # Retrieve the solver runtime
            #model.pprint()
            #return [pyo.value(model.obj), x_values, lam_values, rho_values, y_values, results.solver.time]
            return [pyo.value(model.obj), x_values, results.solver.time]
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM MLE: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)), -1]



In [35]:
error_i2 = []
error_j2 = []
for x in range(len(error_i)):
    curr_limit_result = mccm_limit_kl2(all_full_instances[error_i[x]][error_j[x]],all_full_collections[error_i[x]],all_full_offertimes[error_i[x]])
    if curr_limit_result[-1]<0:
        error_i2.append(error_i[x])
        error_j2.append(error_j[x])
        print(f'error in instance ({error_i[x]}, {error_j[x]})')
    else:
        all_mccm_limit_loss[error_i[x]][error_j[x]] = curr_limit_result[0]
        all_mccm_limit_prob[error_i[x]][error_j[x]] = curr_limit_result[1]
        all_mccm_limit_runtime[error_i[x]][error_j[x]] = curr_limit_result[-1]
        print(f'solve KL for instance ({error_i[x]}, {error_j[x]})')


Error: Cannot load a SolverResults object with bad status: error
MCCM MLE: No solution found or an error occurred.
error in instance (0, 1)
solve KL for instance (0, 5)
solve KL for instance (0, 19)
solve KL for instance (0, 23)
solve KL for instance (0, 25)
solve KL for instance (0, 28)
solve KL for instance (0, 31)
solve KL for instance (0, 32)
solve KL for instance (0, 34)
solve KL for instance (0, 37)
solve KL for instance (0, 38)
Error: Cannot load a SolverResults object with bad status: error
MCCM MLE: No solution found or an error occurred.
error in instance (0, 47)
solve KL for instance (1, 0)
solve KL for instance (1, 5)
solve KL for instance (1, 12)
solve KL for instance (1, 19)
solve KL for instance (1, 22)
Error: Cannot load a SolverResults object with bad status: error
MCCM MLE: No solution found or an error occurred.
error in instance (1, 25)
solve KL for instance (1, 28)
solve KL for instance (1, 31)
solve KL for instance (1, 35)
solve KL for instance (1, 40)
solve KL fo

In [99]:


def mccm_limit(data, collection, offertimes):
    #print(f'check the offertimes {offertimes}')
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, initialize=0, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, initialize=0, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, initialize=0, name="abs_val")
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j] * data[i, j] * model.abs_var[i, j]**2 for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    
    not_in_assortment = [[] for _ in range(m)]
    for j in range(m):
        for i in range(n):
            if i not in collection[j]:
                not_in_assortment[j].append(i)
    
    for j in range(m):
        for i in collection[j]:
            model.norm_constraint.add(model.x[i, j] == model.lam[i] + sum(model.y[k, j] * model.rho[k, i] for k in not_in_assortment[j]))
    for j in range(m):
        for i in not_in_assortment[j]:
            model.norm_constraint.add(model.y[i,j] == model.lam[i] + sum(model.y[k, i] * model.rho[k,i] for k in not_in_assortment[j]))
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] == 0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed

    #results = solver.solve(model, tee=True)
    
    #solver.options['max_iter'] = 10000  # Adjust max_iter as needed
    results = solver.solve(model, tee=True)
    status = str(results.solver.status)
    print("Pyomo optimization status:", status)
    
    # Access the optimal solution
    if status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
        model.write('model.mps', io_options={'symbolic_solver_labels': True})
        model.pprint()
    else:
        print('No solution found or an error occurred.')
    
    x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [ model.lam[i].value for i in range(n)]
    lam_values = np.array(lam_values)
    
    rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
    rho_values = np.array(rho_values)
    
    # Retrieve the solver runtime
    runtime = results.solver.time
    print("Solver runtime:", runtime)
    
    #return [value(model.obj), x_values, results.solver.time]
    return [model.obj(), x_values, results.solver.time]
# Example usage:
# mccm_mle(data, collection, offer_times)


In [100]:
def compute_kl_using_l2norm_soln(solution,instance, offertimes):
    kl_loss = 0
    for i in range(solution.shape[0]):
        for j in range(solution.shape[1]):
            if solution[i][j]>0:
                kl_loss = kl_loss - offertimes[j]*instance[i][j]*np.log(solution[i][j])/np.sum(offertimes)
    return kl_loss

In [101]:
for x in range(len(error_i2)):
    l2_norm_limit_result = mccm_limit(all_full_instances[error_i2[x]][error_j2[x]],all_full_collections[error_i2[x]],all_full_offertimes[error_i2[x]])
    projected_kl_loss = compute_kl_using_l2norm_soln(l2_norm_limit_result[1],all_full_instances[error_i2[x]][error_j2[x]],all_full_offertimes[error_i2[x]])
    print(f'solve KL for instance ({error_i2[x]}, {error_j2[x]})')
    all_mccm_limit_loss[error_i2[x]][error_j2[x]] = projected_kl_loss
    all_mccm_limit_prob[error_i2[x]][error_j2[x]] = l2_norm_limit_result[1]
    all_mccm_limit_runtime[error_i2[x]][error_j2[x]] = l2_norm_limit_result[-1]

    

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4392
Number of nonzeros in inequality constraint Jacobian.:      320
Number of nonzeros in Lagrangian Hessian.............:      624

Total number of variables............................:      873
                     variables with only lower bounds:      873
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [102]:
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        if all_mccm_limit_runtime[i][j]<0:
            print(f'error exists in instance ({i}, {j})')

In [109]:
for i in range(len(pred_offer_times_list)): 
    for j in range(pred_instance_size[i]):
        df_mccm_prob = pd.DataFrame(all_mccm_limit_prob[i][j])
        df_mccm_prob.to_csv('limit/mccm_kl/limit_kl/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        

In [110]:
all_mccm_kl_loss = []
all_mccm_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mccm_kl_loss_collection = []
    mccm_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        mccm_prob = pd.read_csv('limit/mccm_kl/limit_kl/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mccm_prob = mccm_prob.drop(mccm_prob.columns[0],axis=1)
        limit_prob = np.array(mccm_prob)
        
        #limit_prob = all_mccm_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        mccm_kl_loss_collection.append(-curr_kl_loss)
        mccm_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    #df_mccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_loss':mccm_kl_loss_collection,'avg_loss':mccm_avg_kl_loss_collection})
    #df_mccm_loss.to_csv('limit/mccm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mccm_kl_loss.append(mccm_kl_loss_collection) 
    all_mccm_avg_kl_loss.append(mccm_avg_kl_loss_collection)
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mccm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mccm_kl_loss[i])/np.sqrt(len(all_mccm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mccm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mccm_avg_kl_loss[i])/np.sqrt(len(all_mccm_avg_kl_loss[i])))
    
df_mccm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mccm_kl_loss':avg_total_kl_loss,'total_mccm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mccm_kl_loss':avg_avg_kl_loss,'avg_mccm_kl_loss_se':avg_avg_kl_loss_se
                                   })
#df_mccm_kl.to_csv('limit/mccm/mccm_kl_summary.csv')
df_mccm_kl 

,offertimes,total_mccm_kl_loss,total_mccm_kl_loss_se,avg_mccm_kl_loss,avg_mccm_kl_loss_se
0,20,20.065426,4.191549,0.002621,0.000548
1,30,198.631151,134.312558,0.026382,0.017839
2,40,9.149025,3.674647,0.001243,0.000499
3,50,5.883304,2.242438,0.000818,0.000312
4,60,3.237031,1.120068,0.000457,0.000158


In [103]:
def deterministic_term(data,offertimes):
    dt = 0 
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] >0:
                dt = dt + offertimes[j]*data[i][j]*np.log(data[i][j])
    return dt,dt/np.sum(offertimes)

In [106]:
def compute_kl_using_soln(solution,instance, offertimes):
    kl_loss = 0
    for i in range(solution.shape[0]):
        for j in range(solution.shape[1]):
            if solution[i][j]>0:
                kl_loss = kl_loss - offertimes[j]*instance[i][j]*np.log(solution[i][j])/np.sum(offertimes)
    return kl_loss

In [107]:
all_mccm_kl_loss = []
all_mccm_avg_kl_loss = []
all_dt = []
all_avg_dt = []

for i in range(len(pred_offer_times_list)):
    
    mccm_kl_loss_collection = []
    mccm_avg_kl_loss_collection = []
    dt_collection = []
    avg_dt_collection = []

    for j in range(pred_instance_size[i]):

        dt,avg_dt = deterministic_term(all_full_instances[i][j],all_full_offertimes[i])
        dt_collection.append(dt)
        avg_dt_collection.append(avg_dt)
        
        curr_kl_loss = compute_kl_using_soln(all_mccm_limit_prob[i][j],all_full_instances[i][j],all_full_offertimes[i])
        
        mccm_kl_loss_collection.append(curr_kl_loss+dt)
        mccm_avg_kl_loss_collection.append(curr_kl_loss/np.sum(all_full_offertimes[i])+ avg_dt)
        
        
    df_mccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_kl_loss':mccm_kl_loss_collection,'avg_mccm_kl_loss':mccm_avg_kl_loss_collection,'mccm_kl_time':all_mccm_limit_runtime[i]})
    df_mccm_loss.to_csv('limit/mccm_kl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')

    all_mccm_kl_loss.append(mccm_kl_loss_collection) 
    all_mccm_avg_kl_loss.append(mccm_avg_kl_loss_collection)
  
    all_dt.append(dt_collection)
    all_avg_dt.append(avg_dt_collection)
    

In [108]:
for i in range(len(pred_offer_times_list)):

    for j in range(pred_instance_size[i]):
        if all_mccm_kl_loss[i][j] <= 0:
            print('check instance',i,j)

check instance 0 0
check instance 0 1
check instance 0 2
check instance 0 3
check instance 0 4
check instance 0 5
check instance 0 6
check instance 0 7
check instance 0 8
check instance 0 9
check instance 0 10
check instance 0 11
check instance 0 12
check instance 0 13
check instance 0 14
check instance 0 15
check instance 0 16
check instance 0 17
check instance 0 18
check instance 0 19
check instance 0 20
check instance 0 21
check instance 0 22
check instance 0 23
check instance 0 24
check instance 0 25
check instance 0 26
check instance 0 27
check instance 0 28
check instance 0 29
check instance 0 30
check instance 0 31
check instance 0 32
check instance 0 33
check instance 0 34
check instance 0 35
check instance 0 36
check instance 0 37
check instance 0 38
check instance 0 39
check instance 0 40
check instance 0 41
check instance 0 42
check instance 0 43
check instance 0 44
check instance 0 45
check instance 0 46
check instance 0 47
check instance 0 48
check instance 0 49
check inst

In [21]:
for i in range(n):

SyntaxError: unexpected EOF while parsing (<ipython-input-21-5c91772e05db>, line 1)

In [ ]:
all_mccm_avg_loss  = []
for i in range(len(pred_offer_times_list)):
    mccm_avg_loss_collection = []
    
    for j in range(pred_instance_size[i]):
        mccm_avg_loss_collection.append(all_mccm_limit_loss[i][j]/np.sum(all_full_offertimes[i]))
    df_mccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_loss':all_mccm_limit_loss[i],'avg_mccm_loss':mccm_avg_loss_collection,'mccm_limit_time':all_mccm_limit_runtime[i]})
    df_mccm_loss.to_csv('limit/mccm/limit/'+str(pred_offer_times_list[i])+'.csv')
    all_mccm_avg_loss.append(mccm_avg_loss_collection)

In [ ]:
def mccm_kl_w_lam(lam,data,collection,offertimes):
    
    n, m = data.shape
    
    ind = [[] for i in range(n)]  ## keeps track of the assortments including product i

    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i)  ## assortment i includes product j
    
    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(n), range(m), domain= pyo.NonNegativeReals)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j not in collection[i]:
                model.norm_constraint.add(model.x[j, i] == 0)
    
    model.mccm_constraint = pyo.ConstraintList()
    for i in range(n):
        for j in range(len(ind[i]) - 1):
            for k in range(j + 1, len(ind[i])):
                if lam[ind[i][j]] >= lam[ind[i][k]]:
                    model.mccm_constraint.add(model.x[i, ind[i][k]] - model.x[i, ind[i][j]]  >= 0)
                if lam[ind[i][j]] <= lam[ind[i][k]]:
                    model.mccm_constraint.add(model.x[i, ind[i][j]] - model.x[i, ind[i][k]]  >= 0)
    
    model.obj = pyo.Objective(expr= -sum( sum(offertimes[j] *data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.minimize)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    
    try:
        # Your optimization solve code here
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            print("Solver found a solution.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            return [pyo.value(model.obj), x_values, results.solver.time]
        
        elif results.solver.status == SolverStatus.warning:
            print("Solver issued a warning.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            return [pyo.value(model.obj), x_values, results.solver.time]
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")
    
    except ValueError as e:
        print(f"Error: {e}")
        
    

In [ ]:
def deterministic_term(data,offertimes):
    dt = 0 
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] >0:
                dt = dt + offertimes[j]*data[i][j]*np.log(data[i][j])
    return dt,dt/np.sum(offertimes)

In [ ]:
all_mccm_kl_loss = []
all_mccm_avg_kl_loss = []
all_mccm_kl_prob = []
all_mccm_2stage_time = []

all_dt = []
all_avg_dt = []

all_total_time = []

for i in range(len(pred_offer_times_list)):
    
    mccm_kl_loss_collection = []
    mccm_avg_kl_loss_collection = []
    dt_collection = []
    avg_dt_collection = []
    mccm_kl_prob_collection = []
    mccm_2stage_time_collection = []
    total_time_collection = []
    
    for j in range(pred_instance_size[i]):
        result = mccm_kl_w_lam(all_mccm_limit_lam[i][j],all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
        # result = [pyo.value(model.obj), x_values, results.solver.time]
        dt,avg_dt = deterministic_term(all_full_instances[i][j],all_full_offertimes[i])
        
        dt_collection.append(dt)
        avg_dt_collection.append(avg_dt)
        
        mccm_kl_loss_collection.append(result[0]+dt)
        mccm_avg_kl_loss_collection.append(result[0]/np.sum(all_full_offertimes[i])+avg_dt)
        
        mccm_kl_prob_collection.append(result[1])
        mccm_2stage_time_collection.append(result[-1])
        
        total_time_collection.append(result[-1] + all_mccm_limit_runtime[i][j])
        
    df_mccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_kl_loss':mccm_kl_loss_collection,'avg_mccm_kl_loss':mccm_avg_kl_loss_collection,'mccm_kl_time':total_time_collection})
    df_mccm_loss.to_csv('limit/mccm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')

    all_mccm_kl_loss.append(mccm_kl_loss_collection) 
    all_mccm_avg_kl_loss.append(mccm_avg_kl_loss_collection)
    all_mccm_kl_prob.append(mccm_kl_prob_collection)
    all_total_time.append(total_time_collection)
        
    all_dt.append(dt_collection)
    all_avg_dt.append(avg_dt_collection)
    

In [ ]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mccm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mccm_kl_loss[i])/np.sqrt(len(all_mccm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mccm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mccm_avg_kl_loss[i])/np.sqrt(len(all_mccm_avg_kl_loss[i])))
    
    avg_runtime.append(np.mean(all_total_time[i]))
    
df_mccm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mccm_kl_loss':avg_total_kl_loss,'total_mccm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mccm_kl_loss':avg_avg_kl_loss,'avg_mccm_kl_loss_se':avg_avg_kl_loss_se,
                                   'mccm_kl_time':avg_runtime
                                   })
df_mccm_kl.to_csv('limit/mccm/mccm_kl_summary.csv')
df_mccm_kl 
    

In [ ]:
for i in range(len(pred_offer_times_list)):
    df_dt = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'dt':all_dt[i],'avg_dt':all_avg_dt[i]})
    df_dt.to_csv('limit/dt/'+str(pred_offer_times_list[i])+'.csv')

In [ ]:
df_mccm_kl = pd.read_csv('limit/mccm/mccm_kl_summary.csv')
df_mccm_kl  = df_mccm_kl.drop(df_mccm_kl.columns[0],axis=1)
df_mccm_kl

In [ ]:
df_lcmccm_kl = pd.read_csv('limit/lcmccm/lcmccm_kl_summary.csv')
df_lcmccm_kl = df_lcmccm_kl.drop(df_lcmccm_kl.columns[0],axis=1)
df_lcmccm_kl

In [ ]:
# Merge the dataframes on the 'offertimes' column
merged_df = df_mccm_kl.merge(df_mccm_kl, on='offertimes').merge(df_lcmccm_kl, on='offertimes')

# Display the merged dataframe
merged_df.to_csv('limit/kl_summary.csv')

In [ ]:
avg_kl_loss_summary = pd.concat([merged_df['offertimes'],
                                merged_df['avg_mccm_kl_loss'],merged_df['avg_mccm_kl_loss_se'],merged_df['mccm_kl_time'],
                                merged_df['avg_mccm_kl_loss'],merged_df['avg_mccm_kl_loss_se'],
                                merged_df['avg_lcmccm_kl_loss'],merged_df['avg_lcmccm_kl_loss_se']]
                                ,axis=1)
avg_kl_loss_summary.to_csv('limit/avg_kl_summary.csv')

In [ ]:
avg_kl_loss_summary

In [ ]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_mccm_kl_prob[i][j])
        df_limit_prob.to_csv('limit/mccm/limit_kl/limit_kl_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [ ]:
all_mccm_kl_loss = []
all_mccm_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mccm_kl_loss_collection = []
    mccm_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        mccm_prob = pd.read_csv('limit/mccm/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mccm_prob = mccm_prob.drop(mccm_prob.columns[0],axis=1)
        limit_prob = np.array(mccm_prob)
        
        #limit_prob = all_mccm_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        mccm_kl_loss_collection.append(-curr_kl_loss)
        mccm_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    #df_mccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_loss':mccm_kl_loss_collection,'avg_loss':mccm_avg_kl_loss_collection})
    #df_mccm_loss.to_csv('limit/mccm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mccm_kl_loss.append(mccm_kl_loss_collection) 
    all_mccm_avg_kl_loss.append(mccm_avg_kl_loss_collection)
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mccm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mccm_kl_loss[i])/np.sqrt(len(all_mccm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mccm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mccm_avg_kl_loss[i])/np.sqrt(len(all_mccm_avg_kl_loss[i])))
    
df_mccm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mccm_kl_loss':avg_total_kl_loss,'total_mccm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mccm_kl_loss':avg_avg_kl_loss,'avg_mccm_kl_loss_se':avg_avg_kl_loss_se
                                   })
#df_mccm_kl.to_csv('limit/mccm/mccm_kl_summary.csv')
df_mccm_kl 

In [ ]:
all_lcmccm_kl_loss = []
all_lcmccm_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    lcmccm_kl_loss_collection = []
    lcmccm_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        lcmccm_prob = pd.read_csv('limit/lcmccm/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        lcmccm_prob = lcmccm_prob.drop(lcmccm_prob.columns[0],axis=1)
        limit_prob = np.array(lcmccm_prob)
        
        #limit_prob = all_lcmccm_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        lcmccm_kl_loss_collection.append(-curr_kl_loss)
        lcmccm_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    #df_lcmccm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'lcmccm_loss':lcmccm_kl_loss_collection,'avg_loss':lcmccm_avg_kl_loss_collection})
    #df_lcmccm_loss.to_csv('limit/lcmccm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_lcmccm_kl_loss.append(lcmccm_kl_loss_collection) 
    all_lcmccm_avg_kl_loss.append(lcmccm_avg_kl_loss_collection)

avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_lcmccm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_lcmccm_kl_loss[i])/np.sqrt(len(all_lcmccm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_lcmccm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_lcmccm_avg_kl_loss[i])/np.sqrt(len(all_lcmccm_avg_kl_loss[i])))
    
df_lcmccm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_lcmccm_kl_loss':avg_total_kl_loss,'total_lcmccm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_lcmccm_kl_loss':avg_avg_kl_loss,'avg_lcmccm_kl_loss_se':avg_avg_kl_loss_se
                                   })
#df_lcmccm_kl.to_csv('limit/lcmccm/lcmccm_kl_summary.csv')
df_lcmccm_kl 

In [ ]:
# independet model kl loss

all_indep_model_kl_loss = []
all_indep_model_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    indep_model_kl_loss_collection = []
    indep_model_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]): 
        idep_model_kl = 0 
        for k in range(len(all_full_collections[i])):
            assortment = all_full_collections[i][k]
            for x in assortment:
                if all_full_instances[i][j][x][k]>0:
                    idep_model_kl = idep_model_kl - all_full_offertimes[i][k]*all_full_instances[i][j][x][k]*np.log((1/len(assortment))/all_full_instances[i][j][x][k])
                    
        indep_model_kl_loss_collection.append(idep_model_kl)
        indep_model_avg_kl_loss_collection.append(idep_model_kl/np.sum(all_full_offertimes[i]))
    df_indep_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'indep_kl_loss':indep_model_kl_loss_collection,'avg_indep_kl_loss':indep_model_avg_kl_loss_collection})
    df_indep_loss.to_csv('limit/indep_model/'+str(pred_offer_times_list[i])+'.csv')
        
    all_indep_model_kl_loss.append(indep_model_kl_loss_collection)
    all_indep_model_avg_kl_loss.append(indep_model_avg_kl_loss_collection)
    
avg_total_indep_kl_loss = []
avg_total_indep_kl_loss_se = []
avg_avg_indep_kl_loss = []
avg_avg_indep_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_indep_kl_loss.append(np.mean(all_indep_model_kl_loss[i]))
    avg_total_indep_kl_loss_se.append(np.std(all_indep_model_kl_loss[i])/np.sqrt(len(all_indep_model_kl_loss[i])))
    
    avg_avg_indep_kl_loss.append(np.mean(all_indep_model_avg_kl_loss[i]))
    avg_avg_indep_kl_loss_se.append(np.std(all_indep_model_avg_kl_loss[i])/np.sqrt(len(all_indep_model_avg_kl_loss[i])))
    
df_indep_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_indep_kl_loss':avg_total_indep_kl_loss,'total_indep_kl_loss_se':avg_total_indep_kl_loss_se,
                                   'avg_indep_kl_loss':avg_avg_indep_kl_loss,'avg_indep_kl_loss_se':avg_avg_indep_kl_loss_se
                                   })
df_indep_kl.to_csv('limit/indep_model/indep_kl_summary.csv')
df_indep_kl         

In [ ]:
# compute the improvement 
# report loss redution compared to independent model
all_mccm_percent = []
all_mccm_percent = []
all_lcmccm_percent = []

mccm_mean_lst = []
mccm_mean_lst = []
lcmccm_mean_lst = []

mccm_se_lst = []
mccm_se_lst = []
lcmccm_se_lst = []

for i in range(len(pred_offer_times_list)): 
    mccm_percent_collection = []
    mccm_percent_collection = []
    lcmccm_percent_collection = []
    for j in range(pred_instance_size[i]): 
        mccm_percent_collection.append(1-(all_mccm_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        mccm_percent_collection.append(1-(all_mccm_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        lcmccm_percent_collection.append(1-(all_lcmccm_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        
    df_percent = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_percent':mccm_percent_collection,'mccm_percent':mccm_percent_collection,'lcmccm_percent':lcmccm_percent_collection})
    df_percent.to_csv('limit/kl_compare/'+str(pred_offer_times_list[i])+'.csv')
    
    mccm_mean_lst.append(np.mean(mccm_percent_collection))
    mccm_mean_lst.append(np.mean(mccm_percent_collection))
    lcmccm_mean_lst.append(np.mean(lcmccm_percent_collection))
    
    mccm_se_lst.append(np.std(mccm_percent_collection)/np.sqrt(len(mccm_percent_collection)))
    mccm_se_lst.append(np.std(mccm_percent_collection)/np.sqrt(len(mccm_percent_collection)))
    lcmccm_se_lst.append(np.std(lcmccm_percent_collection)/np.sqrt(len(lcmccm_percent_collection)))

df_percent_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'mccm_percent':mccm_mean_lst,'mccm_se':mccm_se_lst,
                                   'mccm_percent':mccm_mean_lst,'mccm_se':mccm_se_lst,
                                   'lcmccm_percent':lcmccm_mean_lst,'lcmccm_se':lcmccm_se_lst})
df_percent_summary.to_csv('limit/kl_compare/improve_percent_summary.csv')
df_percent_summary